# Assignment 1
## Ferry Model | Group 14
### Imports

In [1]:
import salabim as sim, numpy as np, pandas as pd
import time, sys, random


## Time Data

In [93]:
CAR_NUMBERS = pd.read_csv("TimeTable.csv", sep=";")

## Variables

In [3]:
SAILING_TIME = sim.Triangular(10,18,13)
NUMBER_OF_CARS = sim.Triangular(70,80,75)
PAYMENT_TIME = sim.Triangular(1,4,2)
LOADING_TIME = sim.Exponential(1/6) # 10 seconds per car
UNLOADING_TIME = sim.Exponential(5/60) # 5 seconds per time
WAITING_TIME_PREPAID = sim.Exponential(0.5) # minutes
PERCENTAGE_PREPAID = 0.2 

SIM_TIME = 60*24 # Time in minutes
REPLICATIONS = 10 # Number of experiment replications

## Components
### Class Car

In [30]:
class Car(sim.Component):
    def setup(self, cartype, paid, location):
        self.cartype = cartype # either tourist or employee
        self.paid = paid # true for prepaid, false for not prepaid
        self.location = location # either mainland or island

    def process(self):  
        # Go to the assigned booth and line depending on the cartype, prepaid and locations
        ## Employee type
        if self.cartype == "employee":
            if self.location == "mainland":
                self.enter(mainland_line1)
            else: 
                self.enter(island_line1)
        ## Tourists
        else: 
            # In case the tourist has prepaid 
            if self.paid:
                if self.location == "mainland":
                    self.enter(mainland_line2)
                else: 
                    self.enter(island_line2)
            # In case the tourist still has to pay
            else: 
                if self.location == "mainland":
                    self.enter(mainland_line3)
                else: 
                    self.enter(island_line3)
            
        # Passivate the component
        # yield self.passivate()

### Car Generator

In [91]:
class CarGenerator(sim.Component):
    def setup(self, location, cartype):
        self.cartype = cartype # either tourist or employee
        self.location = location # either mainland or island

    def process(self):
        while True:
            #Get current time
            CurrentCarNumbers = CAR_NUMBERS[CAR_NUMBERS["time"]<= (env.now()/60)].tail(1)

            # Wait for the correct amount of time until creating the next car
            ## Get the number of cars
            number_cars = int(CurrentCarNumbers[str(self.cartype) + "_" + str(self.location)])

            ## Calculate the time span the cars arrive in 
            time_span = int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index + 1]) - int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index])

            ## Check if number of cars is greater than zero, then wait the correct amount of time, otherwise wait for the time interval
            if number_cars > 0:
                # Generate a car
                Car(cartype = self.cartype, paid = (random.random() < PERCENTAGE_PREPAID), location= self.location)

                # Interarrival times are based on an exponential equation
                yield self.hold(sim.Exponential(60*time_span / number_cars))

            else:
                yield self.hold(1)
                #yield self.hold(int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index + 1])*60-env.now())



### Ferry

In [107]:
class Ferry(sim.Component):
    def setup(self, capacity, carsonferry, ferryrides, location):
        self.capacity = capacity # indicates how much space there is on the ferry
        self.carsonferry = carsonferry # indicates how many cars there are currently on the ferry 
        self.ferryrides = ferryrides # counts the amount of ferryrides done
        self.location = location # the location of the ferry (either mainland or island)

    # Cruising process of the ferry
    def cruise(self):
        # Set the departure time to False
        departuretime.set(False)

        # Add one more ride to the ferry ride attribute
        self.ferryrides += 1

        # Cruise
        yield self.hold(SAILING_TIME.sample())

        # Change the location of the ferry
        if self.location == "mainland":
            self.location = "island"
        else:
            self.location = "mainland"
        

    # Loading process of the ferry
    def load(self):  
        # Determine the capacity
        self.capacity = NUMBER_OF_CARS.sample()

        # As long as there is space left, check the waiting lines and fill up the space
        while self.capacity > self.carsonferry:
            # Check if it is already departure time
            if departuretime:
                break
            
            # Check location
            if self.location == "mainland":
                # Check if there are any cars left in the queues 
                if len(mainland_line1) > 0:
                    self.car = mainland_line1.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(mainland_line2) > 0:
                    self.car = mainland_line2.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(mainland_line3) > 0:
                    self.car = mainland_line3.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())

                # If no cars are left, set the ferryloaded state to true and check again in 5 mins
                else:
                    ferryloaded.set(True)
                    yield self.hold(5)


            # Same goes for the Island
            if self.location == "island":
                if len(island_line1) > 0:
                    self.car = island_line1.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(island_line2) > 0:
                    self.car = island_line2.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(island_line3) > 0:
                    self.car = island_line3.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                else:
                    ferryloaded.set(True)
                    yield self.hold(5)

        # When the maximum capacity is reached, set the ferry to loaded
        ferryloaded.set(True)

    # Unloading of the ferry
    def unload(self):
        for i in range(self.carsonferry):
            yield self.hold(UNLOADING_TIME.sample())
        self.carsonferry = 0
        ferryloaded.set(False)

            



## Ferry Operator

In [106]:
class FerryOperator(sim.Component):
    def process(self):
        while True:
            # Load the ferry       
            CanadianFerry.activate(process='load')

            # Wait for 30 mins
            yield self.hold(30)

            # Give the go for departure
            departuretime.set(True)

            # Wait until both the departure time is reached and the ferry is loaded
            yield self.wait(departuretime, ferryloaded, all=True)

            # Cruise the ferry
            CanadianFerry.activate(process='cruise')

            # Unload the ferry
            CanadianFerry.activate(process='unload')

### Creating and running the environment

In [108]:
# Create the Environment
env = sim.Environment(time_unit='minutes', trace= True)
env.modelname("Canadian Ferries Simulation")

# Create States
departuretime = sim.State('departuretime', value=False)
ferryloaded = sim.State('ferryloaded', value=False)

# Create Queues
mainland_line1, mainland_line2, mainland_line3 = sim.Queue('mainland_line1'), sim.Queue('mainland_line2'), sim.Queue('mainland_line3')
island_line1, island_line2, island_line3 = sim.Queue('island_line1'), sim.Queue('island_line2'), sim.Queue('island_line3')

# Create a ferry at the beginning of the simulation
CanadianFerry = Ferry(capacity = NUMBER_OF_CARS.sample(), carsonferry = 0, ferryrides = 0, location = "mainland")

# Activate the Ferry operators on 6:30
FerryOperator(at=6.5*60)

# Initiate the Car Generators
CarGenerator(cartype="employee", location="island", at = 6*60)
CarGenerator(cartype="employee", location="mainland", at = 6*60)
CarGenerator(cartype="tourist", location="island", at = 6*60)
CarGenerator(cartype="tourist", location="mainland", at = 6*60)

env.run(duration=SIM_TIME)
print()
island_line3.print_statistics()

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               <ipython-input-108-a107580c72cf>
    2                                  default environment initialize       
    2                                  main create                          
    2       0.000 main                 current                              
    6                                  departuretime create                 value= False
    7                                  ferryloaded create                   value= False
   10                                  mainland_line1 create                
   10                                  mainland_line2 create                
   10                                  mainland_line3 create                
   11                                 

In [103]:
mainland_line3.print_info()

Queue 0x7fb788d33310
  name=mainland_line3
  component(s):
    car.3                enter_time   360.000 priority=0
    car.7                enter_time   380.858 priority=0
    car.10               enter_time   387.472 priority=0
    car.14               enter_time   399.088 priority=0
    car.33               enter_time   444.670 priority=0
    car.41               enter_time   465.812 priority=0
    car.51               enter_time   486.294 priority=0
    car.74               enter_time   535.237 priority=0
    car.81               enter_time   559.483 priority=0
    car.82               enter_time   559.698 priority=0
    car.84               enter_time   565.463 priority=0
    car.85               enter_time   576.460 priority=0
    car.87               enter_time   582.442 priority=0
    car.88               enter_time   582.855 priority=0
    car.92               enter_time   586.878 priority=0
    car.94               enter_time   594.482 priority=0
    car.95               ente

In [23]:
island_line1.pop()

line numbers prefixed by D refer to  <ipython-input-23-7bb00429e3e9>
   D1                                  car.0                                leave island_line1


Car (car.0)

In [43]:
len(island_line1)

80

NameError: name 'ferry' is not defined